In [1]:
%%time

import numpy as np
import torch
import gc
import copy
from torch import nn
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import random
from tqdm.notebook import tqdm
from NN_models import NN_2_256, NN_8_256, NN_8_64, test
from dataset import make_reactions_dict
from reaction_energy_calculation import calculate_reaction_energy


def set_random_seed(seed):
    # seed everything
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(41)

CPU times: user 964 ms, sys: 271 ms, total: 1.24 s
Wall time: 1.49 s


In [2]:
%%time

# make a single dictionary from the whole dataset
data = make_reactions_dict(path='data')

CPU times: user 18.5 s, sys: 7.02 s, total: 25.5 s
Wall time: 26.8 s


In [4]:
data[73] # take the easiest reaction H2 = 2H

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype='<U20'),
 'Coefficients': tensor([ 2., -1.]),
 'Energy': tensor(109.4900),
 'Grid': tensor([[ -1.1678, -16.1181, -16.0818,  ..., -16.1181, -16.1033, -16.1181],
         [ -1.1678, -16.1181, -14.7017,  ..., -16.1181, -15.3051, -16.1181],
         [ -1.1678, -16.1181, -12.3610,  ..., -16.1181, -13.2385, -16.1181],
         ...,
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148],
         [ -7.7005,  -7.7005, -13.7295,  ..., -13.7295,  -8.2041,  -8.2041],
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02]),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.0000],
         ...,
         [0.0003, 0.0003],
         [0.0005, 0.0005],
         [0.0003, 0.0003]]),
 'Gradients': tensor([[3.6975e-09, 3.6975e-09, 0.0000e+00],

In [5]:
def rename_keys(data):
    # turns reaction_data dict keys names into numbers
    l = len(data)
    keys = data.keys()
    data_new = {}
    for i, key in zip(range(l), keys):
        data_new[i] = data[key]
    return data_new


def train_split(data, test_size, shuffle=False):
    # returns train and test reaction dictionaries
    if shuffle:
        keys = list(data.keys())
        random.shuffle(keys)
        for i in keys:
            data[keys[i]] = data[i]

    train, test = dict(), dict()
    border = round(len(data.keys()) * (1 - test_size))
    for i in range(len(data.keys())):
        if i <= border:
            train[i] = data[i]
        else:
            test[i] = data[i]
    return rename_keys(train), rename_keys(test)

In [6]:
test_size = 0.2

data_train, data_test = train_split(copy.deepcopy(data), test_size, shuffle=True)

In [7]:
#standard scaler
lst = []
for i in range(len(data_train)):
    lst.append(data_train[i]['Grid'])

train_grid_data = torch.cat(lst)
print(train_grid_data.shape)

torch.Size([73905233, 7])


In [8]:
%%time
stdscaler = StandardScaler()
stdscaler.fit(np.array(train_grid_data))

CPU times: user 11.6 s, sys: 1.39 s, total: 13 s
Wall time: 13.1 s


StandardScaler()

In [9]:
%%time
for data_t in (data_train, data_test):
    for i in range(len(data_t)):
        data_t[i]['Grid'] = torch.Tensor(stdscaler.transform(data_t[i]['Grid']))

CPU times: user 4.74 s, sys: 80.8 ms, total: 4.82 s
Wall time: 4.86 s


In [10]:
data_test[0]

{'Database': 'NCCE31',
 'Components': array(['HCl-HCl_ncce31', 'HCl_ncce31'], dtype='<U20'),
 'Coefficients': tensor([-1.,  2.]),
 'Energy': tensor(2.0100),
 'Grid': tensor([[1.8193, 1.8079, 1.9052,  ..., 1.9413, 1.8151, 1.8091],
         [1.8193, 1.8079, 2.0157,  ..., 2.0501, 1.8209, 1.8145],
         [1.8193, 1.8078, 2.0798,  ..., 2.1132, 1.8580, 1.8490],
         ...,
         [1.1998, 1.2391, 1.2329,  ..., 1.2795, 1.1805, 1.2183],
         [1.2115, 1.2498, 1.2455,  ..., 1.2918, 1.1918, 1.2289],
         [1.1998, 1.2391, 1.2329,  ..., 1.2795, 1.1805, 1.2183]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.5979e-02, 2.2624e-02,
         2.5979e-02]),
 'Densities': tensor([[1.5987e+03, 1.5987e+03],
         [1.5986e+03, 1.5986e+03],
         [1.5977e+03, 1.5977e+03],
         ...,
         [2.7319e-04, 2.7319e-04],
         [3.6636e-04, 3.6636e-04],
         [2.7319e-04, 2.7319e-04]]),
 'Gradients': tensor([[2.1993e+05, 8.7972e+05, 2.1993e+05],
         [1.8339e+07,

In [2]:
%%time

import pickle
# with open('checkpoints/data_train.pickle', 'wb') as f:
#     pickle.dump(data_train, f)

with open('checkpoints/data_train.pickle', 'rb') as f:
    data_train = pickle.load(f)
with open('checkpoints/data_test.pickle', 'rb') as f:
    data_test = pickle.load(f)

CPU times: user 1.81 s, sys: 2.38 s, total: 4.19 s
Wall time: 4.26 s


In [11]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        # i = 1
        self.data[i].pop('Database', None)

        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())


train_set = Dataset(data=data_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                               batch_size=None,
                                               num_workers=1,
                                               pin_memory=True,
                                               shuffle=True)

test_set = Dataset(data=data_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                              batch_size=None,
                                              num_workers=1,
                                              pin_memory=True,
                                              shuffle=True)

In [24]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')

# model = NN_2_256(nconstants=21, DFT='SVWN').to(device)

model = NN_2_256(nconstants=27, DFT='PBE').to(device)

model.load_state_dict(torch.load('predopt/PBE_2_256_test.param', map_location=device))

<All keys matched successfully>

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3, betas=(0.9, 0.999))

In [14]:
criterion = nn.MSELoss()

In [52]:
from importlib import reload
import NN_models
reload(NN_models)

<module 'NN_models' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/NN_models.py'>

In [46]:
%%time


true_constants_SVWN = [0.0310907, 0.01554535, 
            3.72744,   7.06042,
            12.9352,   18.0578,
            -0.10498,  -0.32500,
            0.0310907,  0.01554535,  -1/(6*np.pi**2),
            13.0720,    20.1231,      1.06835,
            42.7198,   101.578,      11.4813,
            -0.409286,  -0.743294,   -0.228344,
            1]

true_constants_PBE = torch.Tensor([[0.06672455060314922,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709921,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 3.3662,  0.88026,
       0.49294, 0.62517, 0.49671,
       1,  1,  1,
       0.031091, 0.015545, 0.016887,
       0.21370,  0.20548,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])


y_single = true_constants_PBE

class DatasetPredopt(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        i = 0
        self.data[i].pop('Database', None)

        return self.data[i], y_single 
    
    def __len__(self):
        return len(self.data.keys())


train_predopt_set = DatasetPredopt(data=data)
train_predopt_dataloader = torch.utils.data.DataLoader(train_predopt_set,
                                                       batch_size=None,
                                                       num_workers=1,
                                                       pin_memory=True,
                                                       shuffle=True)

CPU times: user 572 µs, sys: 0 ns, total: 572 µs
Wall time: 584 µs


In [47]:
from predopt import predopt


predopt(model, criterion, optimizer, train_predopt_dataloader, device, n_epochs=1)

Epoch 1


  0%|          | 0/267 [00:00<?, ?it/s]

train MSE Loss = 0.13953993
train MAE Loss = 0.07829372


([0.13953992913463223], [0.07829372])

In [17]:
# torch.save(model.state_dict(), 'predopt/PBE_2_256_test.param')

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))

In [19]:
torch.cuda.empty_cache()

In [19]:
from tqdm.notebook import tqdm
import os
log_file_path = 'log/epoch_training.log'

In [20]:
if os.path.isfile(log_file_path):
    os.remove(log_file_path)

In [25]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20, accum_iter=4, verbose=False):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        model.train()
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for batch_idx, (X_batch, y_batch) in enumerate(progress_bar_train):
            
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = model(X_batch_grid)
            # print(predictions)
            # predictions = torch.tile(torch.Tensor(true_constants_PBE), [X_batch_grid.shape[0],1]).to(device, non_blocking=True)
            # print(predictions)
            # print(torch.max(torch.abs(predictions)))
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f}")
            loss = criterion(reaction_energy, y_batch)

            # loss_accumulation
            loss = loss / accum_iter
            
            loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)

                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                train_mse_losses_per_epoch.append(MSE)
                train_mae_losses_per_epoch.append(MAE)
                progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)

                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                    # f.write(f"{preds}\n")
                    
                del MAE, MSE
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy
            gc.collect()
            torch.cuda.empty_cache()
            # gpu_usage()
        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        # progress_bar_test = tqdm(test_loader)
        model.eval()
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                predictions = model(X_batch_grid)
                reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
                loss = criterion(reaction_energy, y_batch)
                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                progress_bar_train.set_postfix(MAE = MAE)
                test_mse_losses_per_epoch.append(MSE)
                test_mae_losses_per_epoch.append(MAE)
                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                
                del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
                gc.collect()
                torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [52]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [28]:
train_loss_mae = train(model, criterion, optimizer,
                              train_dataloader,
                              test_dataloader, n_epochs=10)

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

train MSE Loss = 2327.80249714 MAE Loss = 46.74373307
res_A is inf


ValueError: infinity detected

In [ ]:
train_loss_mae

In [30]:
def test_dft(train_loader, test_loader, c_arr, criterion, rung, dft, verbose=False, n_epochs=1):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_train:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            # print(torch.tile(c_arr, [X_batch_grid.shape[0],1]))
            predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")

            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()

        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        # progress_bar_test = tqdm(test_loader)
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        for X_batch, y_batch in test_loader:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = torch.tile(c_arr, (X_batch_grid.shape[0],1)).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            # progress_bar_train.set_postfix(MAE = MAE)
            test_mse_losses_per_epoch.append(MSE)
            test_mae_losses_per_epoch.append(MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [16]:
from importlib import reload
import NN_models
reload(NN_models)
from NN_models import NN_2_256, NN_8_256, NN_8_64
# import SVWN3

In [37]:
true_constants_SVWN = torch.Tensor([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*torch.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])

true_constants_PBE = torch.Tensor([[0.06672455060314922,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709920934161365617563962776245,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 3.3662,  0.88026,
       0.49294, 0.62517, 0.49671,
       1,  1,  1,
       0.031091, 0.015545, 0.016887,
       0.21370,  0.20548,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])

# true_constants

test_dft(train_dataloader, test_dataloader, true_constants_PBE, criterion, rung='GGA', dft='PBE', verbose=False)
# test_dft(train_dataloader, test_dataloader, true_constants_SVWN, criterion, rung='LDA', dft='SVWN3', verbose=False)


# PBE 45.47375717 35.76542641
# LDA 32.63222432 26.81526929

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

train MSE Loss = 7856.53110129 MAE Loss = 45.47375717
test MSE Loss = 4927.27600202 MAE Loss = 35.76542641


[45.47375716634615]

In [103]:
xs0 = torch.load('log/xs0.pt')
xs1 = torch.load('log/xs1.pt')
xt = torch.load('log/xt.pt')
rho = torch.load('log/rho.pt')
sigmas = torch.load('log/sigmas.pt')

In [36]:
data[73]

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype='<U20'),
 'Coefficients': tensor([ 2., -1.]),
 'Energy': tensor(109.4899978637695312500000000),
 'Grid': tensor([[ -1.1677658557891845703125000, -16.1180953979492187500000000,
          -16.0817871093750000000000000,  ...,
          -16.1180953979492187500000000, -16.1033458709716796875000000,
          -16.1180953979492187500000000],
         [ -1.1677658557891845703125000, -16.1180953979492187500000000,
          -14.7016716003417968750000000,  ...,
          -16.1180953979492187500000000, -15.3050699234008789062500000,
          -16.1180953979492187500000000],
         [ -1.1677662134170532226562500, -16.1180953979492187500000000,
          -12.3609895706176757812500000,  ...,
          -16.1180953979492187500000000, -13.2384977340698242187500000,
          -16.1180953979492187500000000],
         ...,
         [ -8.0003814697265625000000000,  -8.0003814697265625000000000,
          -14.26584720611572265

In [55]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [56]:
calculate_reaction_energy(data[73], torch.tile(true_constants_PBE, [data[73]['Grid'].shape[0],1]).to(device), device, rung='GGA', dft='PBE')

tensor([-0.0574636682868003845214844, -0.0574636720120906829833984,
        -0.0574636645615100860595703,  ...,
        -0.0582121163606643676757812, -0.0564256645739078521728516,
        -0.0582121163606643676757812], device='cuda:0')


tensor(82.9000549316406250000000000, device='cuda:0')

In [45]:
import pickle
with open('checkpoints/calc_reaction_data.pickle', 'rb') as f:
    calc_reaction_data = pickle.load(f)
with open('checkpoints/reaction.pickle', 'rb') as f:
    reaction = pickle.load(f)  

In [47]:
calc_reaction_data

{'Local_energies': tensor([-0.6879517436027526855468750, -0.6879517436027526855468750,
         -0.6879518628120422363281250,  ...,
         -0.1652928292751312255859375, -0.1730412542819976806640625,
         -0.1652928292751312255859375], device='cuda:0'),
 'Densities': tensor([[0.3110610246658325195312500, 0.0000000000000000000000000],
         [0.3110610246658325195312500, 0.0000000000000000000000000],
         [0.3110609054565429687500000, 0.0000000000000000000000000],
         ...,
         [0.0003352346247993409633636, 0.0003352346247993409633636],
         [0.0004524962278082966804504, 0.0004524962278082966804504],
         [0.0003352346247993409633636, 0.0003352346247993409633636]],
        device='cuda:0'),
 'Weights': tensor([2.5717263619955344411058262e-17, 9.9780100021730315507095810e-15,
         3.2611004283263078651344813e-13,  ...,
         2.6054665446281433105468750e-02, 2.2679904475808143615722656e-02,
         2.6054665446281433105468750e-02], device='cuda:0')}

In [104]:
xs0.isnan().sum()

tensor(434, device='cuda:0')

In [105]:
xs1.isnan().sum()

tensor(434, device='cuda:0')

In [106]:
xt.isnan().sum()

tensor(434, device='cuda:0')

In [107]:
rho.isnan().sum()

tensor(0, device='cuda:0')

In [108]:
sigmas.isnan().sum()

tensor(0, device='cuda:0')

In [112]:
DIMENSIONS = 3
eps = 1e-29
xs1_test = torch.where((sigmas[:,2] < eps) & (rho[:,1] < eps), # last sigma and last rho equal 0
                  torch.sqrt(sigmas[:,0])/rho[:,0]**(1 + 1/DIMENSIONS), 
                  torch.sqrt(sigmas[:,2])/rho[:,1]**(1 + 1/DIMENSIONS))

In [114]:
xs1_test.isnan().sum()

tensor(434, device='cuda:0')

In [120]:
(torch.sqrt(sigmas[:,2]))[(torch.sqrt(sigmas[:,0])/rho[:,0]**(1 + 1/DIMENSIONS)).isnan()]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [169]:
(sigmas[:,2] == 0).sum()

tensor(1302, device='cuda:0')

In [147]:
DIMENSIONS = 3

In [150]:
(torch.sqrt(sigmas[:,2])/rho[:,1]**(1 + 1/DIMENSIONS)).isnan().sum()

tensor(434, device='cuda:0')

In [153]:
(rho[:,1]**(1 + 1/DIMENSIONS)).isnan().sum()

tensor(0, device='cuda:0')

tensor(434, device='cuda:0')

In [155]:
(torch.sqrt(sigmas[:,2]))[(torch.sqrt(sigmas[:,2])/rho[:,1]**(1 + 1/DIMENSIONS)).isnan()]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [157]:
(rho[:,1])[(torch.sqrt(sigmas[:,2])/rho[:,1]**(1 + 1/DIMENSIONS)).isnan()]

tensor([3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-36, 3.7042367537315712554845936e-36,
        3.7042367537315712554845936e-3

In [22]:
log.shape

torch.Size([394176])

In [32]:
res_f2.isnan().sum()

tensor(63)

In [33]:
mbeta  = 0.06672455060314922
mgamma = (1 - torch.log(torch.Tensor([2])))/(torch.pi**2)

In [35]:
res_f2_new = mbeta*f1_/(mgamma*(A_1)*f1_)

In [40]:
A_1[res_f2.isnan()]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [41]:
A_1

tensor([1.0352594852447509765625000, 1.0352596044540405273437500,
        1.0352602005004882812500000,  ...,
        1.1321227550506591796875000, 1.1538784503936767578125000,
        1.1526243686676025390625000])

In [51]:
densities = data[73]['Densities']

In [57]:
condition = (densities[:, 0] == 0) & (densities[:, 0] == 0)

In [60]:
densities[torch.logical_not(condition)]

tensor([[0.3110610246658325195312500, 0.0000000000000000000000000],
        [0.3110610246658325195312500, 0.0000000000000000000000000],
        [0.3110609054565429687500000, 0.0000000000000000000000000],
        ...,
        [0.0003352346247993409633636, 0.0003352346247993409633636],
        [0.0004524962278082966804504, 0.0004524962278082966804504],
        [0.0003352346247993409633636, 0.0003352346247993409633636]])

In [28]:
c_arr = torch.rand([30, 30])

In [34]:
c_arr[:, 21:24][:, 1]

tensor([0.1502285599708557128906250, 0.5830305218696594238281250,
        0.5014850497245788574218750, 0.8293408751487731933593750,
        0.7447895407676696777343750, 0.5668825507164001464843750,
        0.8410924673080444335937500, 0.2260936498641967773437500,
        0.0402220487594604492187500, 0.4239581227302551269531250,
        0.8237568140029907226562500, 0.7643668055534362792968750,
        0.2284420728683471679687500, 0.4776555895805358886718750,
        0.7411935925483703613281250, 0.0978116989135742187500000,
        0.6150720119476318359375000, 0.6752546429634094238281250,
        0.4563951492309570312500000, 0.6024737954139709472656250,
        0.9505222439765930175781250, 0.9367821812629699707031250,
        0.3959974646568298339843750, 0.3070790767669677734375000,
        0.1345978379249572753906250, 0.5169637799263000488281250,
        0.8828852772712707519531250, 0.5466136932373046875000000,
        0.5866019725799560546875000, 0.4392346739768981933593750])

In [49]:
y_PBE = torch.Tensor([[0.06672455060314922,
(1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
1.709921,
7.5957, 14.1189, 10.357,
3.5876, 6.1977, 3.6231,
1.6382, 3.3662,  0.88026,
0.49294, 0.62517, 0.49671,
1,  1,  1,
0.031091, 0.015545, 0.016887,
0.21370,  0.20548,  0.11125,
-3/8*(3/torch.pi)**(1/3)*4**(2/3),
0.8040,
0.2195149727645171]])

In [93]:
xs1 = torch.load('xs1.pt')

In [94]:
xs1.isnan().sum()

tensor(33834, device='cuda:0')

In [95]:
xs1

tensor([5.5244732648134231567382812e-03, 5.0763797014951705932617188e-02,
        1.8567425012588500976562500e-01,  ...,
        3.1317285537719726562500000e+01, 1.0731460571289062500000000e+01,
        1.1295479774475097656250000e+01], device='cuda:0')